In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Aula Interativa").getOrCreate()
spark.version

'3.2.0'

In [19]:
titanic_df = spark.read.csv('train.csv', header = 'True', inferSchema = 'True')

In [20]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [21]:
titanic_df.count()

891

In [22]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [23]:
titanic_df.select('Survived','Pclass','Embarked').show()

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       S|
|       0|     3|       S|
|       1|     2|       S|
|       0|     3|       Q|
|       1|     2|       S|
|       0|     3|       S|
|       1|     3|       C|
+--------+------+--------+
only showing top 20 rows



In [24]:
titanic_df.describe('Age','Fare').show()

+-------+------------------+-----------------+
|summary|               Age|             Fare|
+-------+------------------+-----------------+
|  count|               714|              891|
|   mean| 29.69911764705882| 32.2042079685746|
| stddev|14.526497332334035|49.69342859718089|
|    min|              0.42|              0.0|
|    max|              80.0|         512.3292|
+-------+------------------+-----------------+



In [25]:
titanic_df.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [26]:
titanic_df.groupBy('Survived','Sex').count().show()

+--------+------+-----+
|Survived|   Sex|count|
+--------+------+-----+
|       0|female|   81|
|       1|  male|  109|
|       1|female|  233|
|       0|  male|  468|
+--------+------+-----+



In [27]:
from pyspark.sql.functions import col, regexp_extract
titanic_df = titanic_df.withColumn("Initial", regexp_extract(col("Name"),"([A-Za-z]+\.)",1))
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Initial|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr.|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs.|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss.|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs.|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|    Mr.|
|          6|   

In [28]:
titanic_df.select("Initial").distinct().show()

+---------+
|  Initial|
+---------+
|    Miss.|
|      Ms.|
|     Sir.|
|     Rev.|
|      Mr.|
|   Major.|
|     Mrs.|
|      Dr.|
|     Col.|
|  Master.|
|    Mlle.|
|Jonkheer.|
|     Mme.|
|Countess.|
|    Lady.|
|    Capt.|
|     Don.|
+---------+



In [29]:
titanic_df.groupBy('Initial').avg('Age').show()

+---------+------------------+
|  Initial|          avg(Age)|
+---------+------------------+
|    Miss.|21.773972602739725|
|      Ms.|              28.0|
|     Sir.|              49.0|
|     Rev.|43.166666666666664|
|      Mr.|32.368090452261306|
|   Major.|              48.5|
|     Mrs.|35.898148148148145|
|      Dr.|              42.0|
|     Col.|              58.0|
|  Master.| 4.574166666666667|
|    Mlle.|              24.0|
|Jonkheer.|              38.0|
|     Mme.|              24.0|
|Countess.|              33.0|
|    Lady.|              48.0|
|    Capt.|              70.0|
|     Don.|              40.0|
+---------+------------------+



In [30]:
titanic_df.groupBy("Embarked").count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  644|
|    null|    2|
+--------+-----+



In [31]:
titanic_df_ = titanic_df.na.fill({"Embarked":"S"})

In [33]:
titanic_df_.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|       C|  168|
|       S|  646|
+--------+-----+

